# House price prediction using multiple regression analysis

In [1]:
import numpy as np # NumPy is the fundamental package for scientific computing

import pandas as pd # Pandas is an easy-to-use data structures and data analysis tools
pd.set_option('display.max_columns', None) # To display all columns

import matplotlib.pyplot as plt # Matplotlib is a python 2D plotting library
%matplotlib inline 
# A magic command that tells matplotlib to render figures as static images in the Notebook.

import seaborn as sns # Seaborn is a visualization library based on matplotlib (attractive statistical graphics).
sns.set_style('whitegrid') # One of the five seaborn themes
import warnings
warnings.filterwarnings('ignore') # To ignore some of seaborn warning msg

from scipy import stats

from sklearn import linear_model # Scikit learn library that implements generalized linear models
from sklearn import neighbors # provides functionality for unsupervised and supervised neighbors-based learning methods
from sklearn.metrics import mean_squared_error # Mean squared error regression loss
from sklearn import preprocessing # provides functions and classes to change raw feature vectors

from math import log

/Users/adhiman/anaconda3/envs/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
data = pd.read_csv("data/kc_house_data.csv.zip", parse_dates = ['date']) # load the data into a pandas dataframe
data.head(2) # Show the first 2 lines

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639


In [3]:
data.drop(['id', 'date'], axis = 1, inplace = True)

In [4]:
data['basement_present'] = data['sqft_basement'].apply(lambda x: 1 if x > 0 else 0) # Indicate whether there is a basement or not
data['renovated'] = data['yr_renovated'].apply(lambda x: 1 if x > 0 else 0) # 1 if the house has been renovated

In [5]:
categorial_cols = ['floors', 'view', 'condition', 'grade']

for cc in categorial_cols:
    dummies = pd.get_dummies(data[cc], drop_first=False)
    dummies = dummies.add_prefix("{}#".format(cc))
    data.drop(cc, axis=1, inplace=True)
    data = data.join(dummies)

In [6]:
dummies_zipcodes = pd.get_dummies(data['zipcode'], drop_first=False)
dummies_zipcodes.reset_index(inplace=True)
dummies_zipcodes = dummies_zipcodes.add_prefix("{}#".format('zipcode'))
dummies_zipcodes = dummies_zipcodes[['zipcode#98004','zipcode#98102','zipcode#98109','zipcode#98112','zipcode#98039','zipcode#98040']]
data.drop('zipcode', axis=1, inplace=True)
data = data.join(dummies_zipcodes)

data.dtypes

price               float64
bedrooms              int64
bathrooms           float64
sqft_living           int64
sqft_lot              int64
waterfront            int64
sqft_above            int64
sqft_basement         int64
yr_built              int64
yr_renovated          int64
lat                 float64
long                float64
sqft_living15         int64
sqft_lot15            int64
basement_present      int64
renovated             int64
floors#1.0            uint8
floors#1.5            uint8
floors#2.0            uint8
floors#2.5            uint8
floors#3.0            uint8
floors#3.5            uint8
view#0                uint8
view#1                uint8
view#2                uint8
view#3                uint8
view#4                uint8
condition#1           uint8
condition#2           uint8
condition#3           uint8
condition#4           uint8
condition#5           uint8
grade#1               uint8
grade#3               uint8
grade#4               uint8
grade#5             

## Split the dataset

In [7]:
from sklearn.cross_validation import train_test_split
train_data, test_data = train_test_split(data, train_size = 0.8, random_state = 10)

/Users/adhiman/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Regression models

### Simple linear regression

In [8]:
# A function that take one input of the dataset and return the RMSE (of the test data), and the intercept and coefficient
def simple_linear_model(train, test, input_feature):
    regr = linear_model.LinearRegression() # Create a linear regression object
    regr.fit(train.as_matrix(columns = [input_feature]), train.as_matrix(columns = ['price'])) # Train the model
    RMSE = mean_squared_error(test.as_matrix(columns = ['price']), 
                              regr.predict(test.as_matrix(columns = [input_feature])))**0.5 # Calculate the RMSE on test data
    return RMSE, regr.intercept_[0], regr.coef_[0][0]

In [9]:
RMSE, w0, w1 = simple_linear_model(train_data, test_data, 'sqft_living')
print ('RMSE for sqft_living is: %s ' %RMSE)
print ('intercept is: %s' %w0)
print ('coefficient is: %s' %w1)

RMSE for sqft_living is: 268279.6438833362 
intercept is: -36738.177346383105
coefficient is: 277.36412987021157


In [10]:
input_list = data.columns.values.tolist() # list of column name
input_list.remove('price')
simple_linear_result = pd.DataFrame(columns = ['feature', 'RMSE', 'intercept', 'coefficient'])

# loop that calculate the RMSE of the test data for each input 
for p in input_list:
    RMSE, w1, w0 = simple_linear_model(train_data, test_data, p)
    simple_linear_result = simple_linear_result.append({'feature':p, 'RMSE':RMSE, 'intercept':w0, 'coefficient': w1}
                                                       ,ignore_index=True)
simple_linear_result.sort_values('RMSE').head(10) # display the 10 best estimators

,feature,RMSE,intercept,coefficient
2,sqft_living,268279.643883,277.364130,-3.673818e+04
5,sqft_above,304131.310592,266.306764,6.461714e+04
11,sqft_living15,320686.541323,314.359911,-8.502590e+04
1,bathrooms,324082.781919,246523.891877,1.863279e+04
21,view#0,356019.001320,-435033.777431,9.322014e+05
6,sqft_basement,357843.745395,258.126523,4.642966e+05
40,grade#11,357964.423743,965286.415396,5.222663e+05
39,grade#10,360773.700418,556992.601325,5.107024e+05
0,bedrooms,361295.375626,117579.891853,1.436815e+05
9,lat,365041.433662,814499.981062,-3.819789e+07


### Multiple linear regression

In [11]:
# A function that take multiple features as input and return the RMSE (of the test data), and the  intercept and coefficients
def multiple_regression_model(train, test, input_features):
    regr = linear_model.LinearRegression() # Create a linear regression object
    regr.fit(train.as_matrix(columns = input_features), train.as_matrix(columns = ['price'])) # Train the model
    RMSE = mean_squared_error(test.as_matrix(columns = ['price']), 
                              regr.predict(test.as_matrix(columns = input_features)))**0.5 # Calculate the RMSE on test data
    return RMSE, regr.intercept_[0], regr.coef_ 

In [12]:
print ('RMSE: %s, intercept: %s, coefficients: %s' %multiple_regression_model(train_data, 
                                                                             test_data, ['sqft_living','bathrooms','bedrooms']))
print ('RMSE: %s, intercept: %s, coefficients: %s' %multiple_regression_model(train_data, 
                                                                             test_data, ['sqft_above','view#0','bathrooms']))
print ('RMSE: %s, intercept: %s, coefficients: %s' %multiple_regression_model(train_data, 
                                                                             test_data, ['bathrooms','bedrooms']))
print ('RMSE: %s, intercept: %s, coefficients: %s' %multiple_regression_model(train_data, 
                                                                             test_data, ['view#0','grade#12','bedrooms','sqft_basement']))
print ('RMSE: %s, intercept: %s, coefficients: %s' %multiple_regression_model(train_data, 
                                                                             test_data, ['sqft_living','bathrooms','view#0']))

RMSE: 264872.2835550953, intercept: 81100.95967753517, coefficients: [[   306.15090562   7913.53847651 -57658.90103459]]
RMSE: 282802.3649615518, intercept: 303531.2921883529, coefficients: [[ 1.98928518e+02 -3.17760670e+05  7.92684094e+04]]
RMSE: 323412.2692762603, intercept: -18432.90570721531, coefficients: [[235300.89998266  18030.65120532]]
RMSE: 320893.6584322429, intercept: 507958.8899922919, coefficients: [[-3.31957675e+05  1.35299763e+06  8.56179339e+04  1.24073575e+02]]
RMSE: 260210.04852365193, intercept: 205076.198921312, coefficients: [[    258.00653033    -244.89749378 -223120.61245789]]


In [13]:
train_data['sqft_living_squared'] = train_data['sqft_living'].apply(lambda x: x**2) # create a new column in train_data
test_data['sqft_living_squared'] = test_data['sqft_living'].apply(lambda x: x**2) # create a new column in test_data
print ('RMSE: %s, intercept: %s, coefficients: %s' %multiple_regression_model(train_data, 
                                                                             test_data, ['sqft_living','sqft_living_squared']))

RMSE: 246063.95920692245, intercept: 172106.56010562018, coefficients: [[9.31721994e+01 3.37454932e-02]]


**Which features to select?**

Lets use a greedy technique like a forward stepwise algorithm where the best estimator feature is added to the set of already selected features at each iteration. For example, let's pretend that the best single estimator is `sqft_living`. In the 2nd step of the greedy algorithm, we test all the remaining features one by one in combinations with `sqft_living` (e.g., `sqft_living` and `bedrooms`, `sqft_living` and `waterfront`, etc) and select the best combination using training error. 

At the end, we select the model complexity (number of features) using the validation error and estimate the generalization error using the test set.

In [14]:
# we're first going to add more features into the dataset.

# sqft_living cubed
train_data['sqft_living_cubed'] = train_data['sqft_living'].apply(lambda x: x**3) 
test_data['sqft_living_cubed'] = test_data['sqft_living'].apply(lambda x: x**3) 

# bedrooms_squared: this feature will mostly affect houses with many bedrooms.
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2) 
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

# bedrooms times bathrooms gives what's called an "interaction" feature. It is large when both of them are large.
train_data['bed_bath_rooms'] = train_data['bedrooms']*train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms']*test_data['bathrooms']

# Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

train_data.shape

(17290, 55)

In [15]:
# split the train_data to include a validation set (train_data2 = 60%, validation_data = 20%, test_data = 20%)
train_data_2, validation_data = train_test_split(train_data, train_size = 0.75, random_state = 50)

In [16]:
# A function that take multiple features as input and return the RMSE (of the train and validation data)
def RMSE(train, validation, features, new_input):
    features_list = list(features)
    features_list.append(new_input)
    regr = linear_model.LinearRegression() # Create a linear regression object
    regr.fit(train.as_matrix(columns = features_list), train.as_matrix(columns = ['price'])) # Train the model
    RMSE_train = mean_squared_error(train.as_matrix(columns = ['price']), 
                              regr.predict(train.as_matrix(columns = features_list)))**0.5 # Calculate the RMSE on train data
    RMSE_validation = mean_squared_error(validation.as_matrix(columns = ['price']), 
                              regr.predict(validation.as_matrix(columns = features_list)))**0.5 # Calculate the RMSE on train data
    return RMSE_train, RMSE_validation 

In [17]:
input_list = train_data_2.columns.values.tolist() # list of column name
input_list.remove('price')

# list of features included in the regression model and the calculated train and validation errors (RMSE)
regression_greedy_algorithm = pd.DataFrame(columns = ['feature', 'train_error', 'validation_error'])  
i = 0
temp_list = []

# a while loop going through all the features in the dataframe
while i < len(train_data_2.columns)-1:
    
    # a temporary dataframe to select the best feature at each iteration
    temp = pd.DataFrame(columns = ['feature', 'train_error', 'validation_error'])
    
    # a for loop to test all the remaining features
    for p in input_list:
        RMSE_train, RMSE_validation = RMSE(train_data_2, validation_data, temp_list, p)
        temp = temp.append({'feature':p, 'train_error':RMSE_train, 'validation_error':RMSE_validation}, ignore_index=True)
        
    temp = temp.sort_values('train_error') # select the best feature using train error
    best = temp.iloc[0,0]
    temp_list.append(best)
    regression_greedy_algorithm = regression_greedy_algorithm.append({'feature': best, 
                                                  'train_error': temp.iloc[0,1], 'validation_error': temp.iloc[0,2]}, 
                                                 ignore_index=True) # add the feature to the dataframe
    input_list.remove(best) # remove the best feature from the list of available features
    i += 1
regression_greedy_algorithm

,feature,train_error,validation_error
0,sqft_living_squared,249721.440480,268747.213950
1,lat,229419.062478,252069.707424
2,waterfront,219240.069871,241979.268406
3,zipcode#98004,208411.508254,229000.747130
4,zipcode#98112,202604.911359,225124.826562
5,zipcode#98039,196885.794407,217328.215296
6,view#0,191119.193990,209679.561580
7,zipcode#98040,186849.129730,204450.501487
8,sqft_living15,185134.594156,197733.047694
9,grade#13,183383.653232,197321.800871


In [18]:
greedy_algo_features_list = regression_greedy_algorithm['feature'].tolist()[:24] # select the first 30 features
test_error, _, _ = multiple_regression_model(train_data_2, validation_data, greedy_algo_features_list)
print ('test error (RMSE) is: %s' %test_error)

test error (RMSE) is: 167570.14770428356


### Ridge regression

In [19]:
input_feature = train_data.columns.values.tolist() # list of column name
input_feature.remove('price')

for i in [1,10]:
    ridge = linear_model.Ridge(alpha = i, normalize = True) # initialize the model
    ridge.fit(train_data.as_matrix(columns = input_feature), train_data.as_matrix(columns = ['price'])) # fit the train data
    print ('test error (RMSE) is: %s' %mean_squared_error(test_data.as_matrix(columns = ['price']), 
                              ridge.predict(test_data.as_matrix(columns = input_feature)))**0.5) # predict price and test error

test error (RMSE) is: 190153.37221800248
test error (RMSE) is: 287976.6316505086


In [20]:
ridgeCV = linear_model.RidgeCV(alphas = np.linspace(1.0e-10,1,num = 100), normalize = True, store_cv_values = True) # initialize the model
ridgeCV.fit(train_data.as_matrix(columns = input_feature), train_data.as_matrix(columns = ['price'])) # fit the train data
print ('best alpha is: %s' %ridgeCV.alpha_) # get the best alpha
print ('test error (RMSE) is: %s' %mean_squared_error(test_data.as_matrix(columns = ['price']), 
                              ridgeCV.predict(test_data.as_matrix(columns = input_feature)))**0.5) # predict price and test error

best alpha is: 0.0707070708
test error (RMSE) is: 171566.79185920305


### Lasso regression

In [21]:
for i in [0.01,0.1,1,250,500,1000]:
    lasso = linear_model.Lasso(alpha = i, normalize = True) # initialize the model
    lasso.fit(train_data.as_matrix(columns = input_feature), train_data.as_matrix(columns = ['price'])) # fit the train data
    print (lasso.sparse_coef_.getnnz) # number of non zero weights
    print ('test error (RMSE) is: %s' %mean_squared_error(test_data.as_matrix(columns = ['price']), 
                              lasso.predict(test_data.as_matrix(columns = input_feature)))**0.5) # predict price and test error

<bound method _cs_matrix.getnnz of <1x54 sparse matrix of type '<class 'numpy.float64'>'
	with 53 stored elements in Compressed Sparse Row format>>
test error (RMSE) is: 174601.68260073222
<bound method _cs_matrix.getnnz of <1x54 sparse matrix of type '<class 'numpy.float64'>'
	with 50 stored elements in Compressed Sparse Row format>>
test error (RMSE) is: 174578.55878672577
<bound method _cs_matrix.getnnz of <1x54 sparse matrix of type '<class 'numpy.float64'>'
	with 45 stored elements in Compressed Sparse Row format>>
test error (RMSE) is: 174412.93966589426
<bound method _cs_matrix.getnnz of <1x54 sparse matrix of type '<class 'numpy.float64'>'
	with 14 stored elements in Compressed Sparse Row format>>
test error (RMSE) is: 203694.35400709457
<bound method _cs_matrix.getnnz of <1x54 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>>
test error (RMSE) is: 241504.25938546794
<bound method _cs_matrix.getnnz of <1x54 sparse matrix o

In [22]:
lassoCV = linear_model.LassoCV(normalize = True) # initialize the model (alphas are set automatically)
lassoCV.fit(train_data.as_matrix(columns = input_feature), np.ravel(train_data.as_matrix(columns = ['price']))) # fit the train data
print ('best alpha is: %s' %lassoCV.alpha_) # get the best alpha
print ('number of non zero weigths is: %s' %np.count_nonzero(lassoCV.coef_)) # number of non zero weights
print ('test error (RMSE) is: %s' %mean_squared_error(test_data.as_matrix(columns = ['price']), 
                              lassoCV.predict(test_data.as_matrix(columns = input_feature)))**0.5) # predict price and test error

best alpha is: 14.936078017798543
number of non zero weigths is: 39
test error (RMSE) is: 171369.07571302887


### KNN Regression

In [23]:
# normalize the data
train_X = train_data.as_matrix(columns = input_feature)
scaler = preprocessing.StandardScaler().fit(train_X)
train_X_scaled = scaler.transform(train_X)
test_X = test_data.as_matrix(columns = input_feature)
test_X_scaled = scaler.transform(test_X)

knn = neighbors.KNeighborsRegressor(n_neighbors=10, weights='distance') # initialize the model
knn.fit(train_X_scaled, train_data.as_matrix(columns = ['price'])) # fit the train data
print ('test error (RMSE) is: %s' %mean_squared_error(test_data.as_matrix(columns = ['price']), 
                              knn.predict(test_X_scaled))**0.5) # predict price and test error

test error (RMSE) is: 184936.00865427667
